# New sheep background data
These are new sheep background data which integrate SMARTER-database

* [Welsh sheep breeds](#welsh_breeds)
* [European mouflon and domestic sheep](#barbato_2017)
* [European and balcan sheeps](#ciani_2020)
* [Northwest Africa sheeps](#northwest_africa)

In [1]:
import re
import logging
from collections import defaultdict

import numpy as np
import pandas as pd
import pycountry
from tqdm.notebook import tqdm
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.point import Point


from src.features.smarterdb import global_connection, Dataset, SampleSheep
from src.features.plinkio import TextPlinkIO, BinaryPlinkIO, CodingException
from src.data.common import WORKING_ASSEMBLIES

In [2]:
logger = logging.getLogger('src.features.plinkio')
logger.setLevel(logging.CRITICAL)
tqdm.pandas()

_ = global_connection()
OAR3 = WORKING_ASSEMBLIES["OAR3"]

In [3]:
class CustomMixin():
    n_of_individuals = None
    
    def process_pedfile(self, coding="top"):
        for line in tqdm(self.read_pedfile(), total=self.n_of_individuals):
            _ = self._process_genotypes(line, coding)
            
        return True
    
    def is_top(self):
        try:
            return self.process_pedfile(coding='top')
        
        except CodingException:
            return False
    
    def is_forward(self):
        try:
            return self.process_pedfile(coding='forward')
        
        except CodingException:
            return False
        
    def is_affymetrix(self):
        try:
            return self.process_pedfile(coding='affymetrix')
        
        except CodingException:
            return False
        
class CustomTextPlinkIO(CustomMixin, TextPlinkIO):
    pass

class CustomBinaryPlinkIO(CustomMixin, BinaryPlinkIO):
    def process_pedfile(self, coding="top"):
        for line in tqdm(self.read_pedfile(), total=len(self.plink_file.get_samples())):
            _ = self._process_genotypes(line, coding)
            
        return True

<a id='welsh_breeds'></a>
## Welsh sheep breeds
This dataset comes from [Beynon, Sarah E. et al. (2016)](https://bmcgenomdata.biomedcentral.com/articles/10.1186/s12863-015-0216-x), in which they genotyped 353 individuals from 18 native Welsh sheep breeds using the Illumina OvineSNP50 array:

In [4]:
welsh_dataset = Dataset.objects.get(file="Welsh_sheep_genotyping.zip")
welsh_dataset.contents

['genotyping data/',
 'genotyping data/WelshSheepBreeds2015.map',
 'genotyping data/WelshSheepBreeds2015.ped',
 'welsh-metadata.openrefine.tar.gz',
 'welsh-metadata.xlsx']

Ok open dataset and start exploring data:

In [5]:
prefix = str(welsh_dataset.working_dir / "genotyping data/WelshSheepBreeds2015")
plinkio = CustomTextPlinkIO(prefix=prefix, species=welsh_dataset.species, chip_name=welsh_dataset.chip_name)
plinkio.n_of_individuals = welsh_dataset.n_of_individuals

In [6]:
plinkio.read_mapfile()
plinkio.fetch_coordinates(src_assembly=OAR3)

In [7]:
snps_found = len(plinkio.mapdata)-len(plinkio.filtered)
perc_missing = round(100 - (snps_found / len(plinkio.mapdata) * 100), 2)

print(f"I can retrieve {snps_found} of {len(plinkio.mapdata)} SNPs using 'name' ({perc_missing}% missing)")

I can retrieve 51135 of 51135 SNPs using 'name' (0.0% missing)


Is this dataset in top coordinates?

In [8]:
plinkio.is_top()

  0%|          | 0/353 [00:00<?, ?it/s]

True

Good. This file is already in *top* coordinates. What about breeds?

In [9]:
breeds = set()

for line in plinkio.read_pedfile():
    breed = line[0]
    if breed not in breeds:
        breeds.add(breed)
    
print(f"Got {breeds} breeds")

Got {'TregaronWelshMountain', 'BrecknockHillCheviot', 'ClunForest', 'TalybontWelshMountain', 'WelshMountainHillFlock', 'Beulah', 'BlackWelshMountain', 'Lleyn', 'DolgellauWelshMountain', 'ImprovedWelshMountain', 'SouthWalesWelshMountain', 'Llawenog', 'BadgerFaced', 'Balwen', 'HillRadnor', 'Llanwenog', 'LlandoveryWhiteFaced', 'HardySpeckledFaced', 'KerryHill'} breeds


Try to split breed names:

In [10]:
# https://stackoverflow.com/a/29920015
def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0) for m in matches]

In [11]:
breeds2dict = {}

for breed in breeds:
    breeds2dict[breed] = " ".join(camel_case_split(breed))
    
print(breeds2dict)

{'TregaronWelshMountain': 'Tregaron Welsh Mountain', 'BrecknockHillCheviot': 'Brecknock Hill Cheviot', 'ClunForest': 'Clun Forest', 'TalybontWelshMountain': 'Talybont Welsh Mountain', 'WelshMountainHillFlock': 'Welsh Mountain Hill Flock', 'Beulah': 'Beulah', 'BlackWelshMountain': 'Black Welsh Mountain', 'Lleyn': 'Lleyn', 'DolgellauWelshMountain': 'Dolgellau Welsh Mountain', 'ImprovedWelshMountain': 'Improved Welsh Mountain', 'SouthWalesWelshMountain': 'South Wales Welsh Mountain', 'Llawenog': 'Llawenog', 'BadgerFaced': 'Badger Faced', 'Balwen': 'Balwen', 'HillRadnor': 'Hill Radnor', 'Llanwenog': 'Llanwenog', 'LlandoveryWhiteFaced': 'Llandovery White Faced', 'HardySpeckledFaced': 'Hardy Speckled Faced', 'KerryHill': 'Kerry Hill'}


Try to create sample metadata table:

In [12]:
data = defaultdict(list)

for line in plinkio.read_pedfile():
    data["breed"].append(breeds2dict[line[0]])
    data["fid"].append(line[0])
    data["original_id"].append(line[1])
    
welsh_metadata = pd.DataFrame(data=data)
welsh_metadata.to_excel("welsh_metadata.xlsx", index=False)

This file will be imported in openrefine in order to fix values and add a breed code for each breeds

<a id='barbato_2017'></a>
## European mouflon and domestic sheep
This dataset comes from [Barbato M, Hailer F, Orozco-terWengel P, et al](https://www.nature.com/articles/s41598-017-07382-7) and have data from *muflon* and sheep

In [13]:
barbato_2017 = Dataset.objects.get(file="41598_2017_7382_MOESM2_ESM.zip")
barbato_2017.contents

['41598_2017_7382_MOESM2_ESM/',
 '41598_2017_7382_MOESM2_ESM/CIWI.R',
 '41598_2017_7382_MOESM2_ESM/Barbato_2016.bim',
 '41598_2017_7382_MOESM2_ESM/Barbato_2016.fam',
 '41598_2017_7382_MOESM2_ESM/PCAdmix_prettify.cpp',
 '41598_2017_7382_MOESM2_ESM/barbato_muflon.xlsx',
 '41598_2017_7382_MOESM2_ESM/barbato_sheep.xlsx',
 '41598_2017_7382_MOESM2_ESM/.Rhistory',
 '41598_2017_7382_MOESM2_ESM/ovis.sample_index',
 '41598_2017_7382_MOESM2_ESM/Barbato_2016.bed',
 '41598_2017_7382_MOESM2_ESM/barbato_muflon_metadata.xlsx',
 '41598_2017_7382_MOESM2_ESM/barbato_sheep_metadata.xlsx']

Ok open dataset and start exploring data:

In [14]:
prefix = str(barbato_2017.working_dir / "41598_2017_7382_MOESM2_ESM/Barbato_2016")
plinkio = CustomBinaryPlinkIO(prefix=prefix, species=barbato_2017.species, chip_name=barbato_2017.chip_name)
plinkio.n_of_individuals = 422

In [15]:
plinkio.read_mapfile()
plinkio.fetch_coordinates(src_assembly=OAR3)

In [16]:
snps_found = len(plinkio.mapdata)-len(plinkio.filtered)
perc_missing = round(100 - (snps_found / len(plinkio.mapdata) * 100), 2)

print(f"I can retrieve {snps_found} of {len(plinkio.mapdata)} SNPs using 'name' ({perc_missing}% missing)")

I can retrieve 36961 of 36961 SNPs using 'name' (0.0% missing)


Is this dataset in top coordinates?

In [17]:
plinkio.is_top()

  0%|          | 0/422 [00:00<?, ?it/s]

True

The entire dataset is in TOP coordinates. It's composed by muflon and by sheep samples. First start by considering muflon samples:

In [18]:
muflon = pd.read_excel(barbato_2017.working_dir / "41598_2017_7382_MOESM2_ESM/barbato_muflon.xlsx")
muflon

,Breed/population,Acronym,Origin,Number,Ne,F,Source,Ho,Ho (SD)
0,Sardinian mouflon,MSar1,Sardinia,19,261.0,0.45,This study,0.22,0.19
1,Sardinian mouflon,MSar2,Sardinia,8,130.0,0.46,This study,0.22,0.24
2,Sardinian mouflon,MSar3,Sardinia,28,273.0,0.16,KJa,0.34,0.19
3,Spanish mouflon,MSpa,Spain,21,96.0,0.51,KJa,0.20,0.19
4,Hungarian mouflon,MHun,Hungary,8,282.0,0.42,This study,0.24,0.21
5,Corsican mouflon,MCor,Corsica,3,259.0,0.41,This study,0.24,0.27
6,Cypriot mouflon,MCyp,Cyprus,3,244.0,0.78,This study,0.09,0.20
7,Iranian mouflon,MIra,Iran,2,NaN,0.35,NGb,0.25,0.31


Replace origin with the proper country:

In [19]:
muflon.replace({"Origin": {"Sardinia": "Italy", "Corsica": "France", "Iran": "Iran, Islamic Republic of"}}, inplace=True)
muflon

,Breed/population,Acronym,Origin,Number,Ne,F,Source,Ho,Ho (SD)
0,Sardinian mouflon,MSar1,Italy,19,261.0,0.45,This study,0.22,0.19
1,Sardinian mouflon,MSar2,Italy,8,130.0,0.46,This study,0.22,0.24
2,Sardinian mouflon,MSar3,Italy,28,273.0,0.16,KJa,0.34,0.19
3,Spanish mouflon,MSpa,Spain,21,96.0,0.51,KJa,0.20,0.19
4,Hungarian mouflon,MHun,Hungary,8,282.0,0.42,This study,0.24,0.21
5,Corsican mouflon,MCor,France,3,259.0,0.41,This study,0.24,0.27
6,Cypriot mouflon,MCyp,Cyprus,3,244.0,0.78,This study,0.09,0.20
7,Iranian mouflon,MIra,"Iran, Islamic Republic of",2,NaN,0.35,NGb,0.25,0.31


Add species:

In [20]:
muflon["Species"] = "Ovis aries musimon"
muflon.at[6,"Species"] = "Ovis orientalis ophion"
muflon.at[7,"Species"] = "Ovis orientalis"
muflon

,Breed/population,Acronym,Origin,Number,Ne,F,Source,Ho,Ho (SD),Species
0,Sardinian mouflon,MSar1,Italy,19,261.0,0.45,This study,0.22,0.19,Ovis aries musimon
1,Sardinian mouflon,MSar2,Italy,8,130.0,0.46,This study,0.22,0.24,Ovis aries musimon
2,Sardinian mouflon,MSar3,Italy,28,273.0,0.16,KJa,0.34,0.19,Ovis aries musimon
3,Spanish mouflon,MSpa,Spain,21,96.0,0.51,KJa,0.20,0.19,Ovis aries musimon
4,Hungarian mouflon,MHun,Hungary,8,282.0,0.42,This study,0.24,0.21,Ovis aries musimon
5,Corsican mouflon,MCor,France,3,259.0,0.41,This study,0.24,0.27,Ovis aries musimon
6,Cypriot mouflon,MCyp,Cyprus,3,244.0,0.78,This study,0.09,0.20,Ovis orientalis ophion
7,Iranian mouflon,MIra,"Iran, Islamic Republic of",2,NaN,0.35,NGb,0.25,0.31,Ovis orientalis


Try to define a breed code to be used in database:

In [21]:
muflon["code"] = muflon["Acronym"].apply(lambda value: value[:3].upper())

rename columns for simplicity:

In [22]:
muflon.rename(columns={"Breed/population": "Breed"}, inplace=True)

Need to define a metadata table in which specify the samples to add:

In [23]:
tmp = defaultdict(list)
for fid, iid, *_ in plinkio.read_pedfile():
    tmp["code"].append(fid)
    tmp["original_id"].append(iid)
            
tmp = pd.DataFrame(data=tmp)
tmp.head()

,code,original_id
0,MCyp,92_MufloneCy
1,MCyp,101_MufloneCy
2,MCyp,113_MufloneCy
3,MIra,MIra-C3-0001
4,MIra,MIra-D6-0003


In [24]:
muflon_metadata = pd.merge(tmp, muflon, left_on="code", right_on="Acronym")
muflon_metadata.rename(columns={"Origin": "country", "code_x": "fid", "code_y": "code"}, inplace=True)
muflon_metadata

,fid,original_id,Breed,Acronym,country,Number,Ne,F,Source,Ho,Ho (SD),Species,code
0,MCyp,92_MufloneCy,Cypriot mouflon,MCyp,Cyprus,3,244.0,0.78,This study,0.09,0.20,Ovis orientalis ophion,MCY
1,MCyp,101_MufloneCy,Cypriot mouflon,MCyp,Cyprus,3,244.0,0.78,This study,0.09,0.20,Ovis orientalis ophion,MCY
2,MCyp,113_MufloneCy,Cypriot mouflon,MCyp,Cyprus,3,244.0,0.78,This study,0.09,0.20,Ovis orientalis ophion,MCY
3,MIra,MIra-C3-0001,Iranian mouflon,MIra,"Iran, Islamic Republic of",2,NaN,0.35,NGb,0.25,0.31,Ovis orientalis,MIR
4,MIra,MIra-D6-0003,Iranian mouflon,MIra,"Iran, Islamic Republic of",2,NaN,0.35,NGb,0.25,0.31,Ovis orientalis,MIR
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,MSar3,SMF24,Sardinian mouflon,MSar3,Italy,28,273.0,0.16,KJa,0.34,0.19,Ovis aries musimon,MSA
88,MSar3,SMF25,Sardinian mouflon,MSar3,Italy,28,273.0,0.16,KJa,0.34,0.19,Ovis aries musimon,MSA
89,MSar3,SMF26,Sardinian mouflon,MSar3,Italy,28,273.0,0.16,KJa,0.34,0.19,Ovis aries musimon,MSA
90,MSar3,SMF27,Sardinian mouflon,MSar3,Italy,28,273.0,0.16,KJa,0.34,0.19,Ovis aries musimon,MSA


There are also additional metadata coming from nextgen for the two iranian samples, which have different names between nextgen and this dataset:

In [25]:
nextgen = pd.read_table(barbato_2017.working_dir / "41598_2017_7382_MOESM2_ESM/ovis.sample_index", skiprows=10)
nextgen.rename(columns={"#sample_name": "alias"}, inplace=True)
nextgen = nextgen[nextgen["alias"].str.contains("C3-0001|D6-0003")].copy()
nextgen

,alias,sample_accession,biosamples_id,sample_provider,species,taxonomy_id,breed,country,closest_city,closest_locality,estimated_age_months,sex,longitude,latitude,sampling_date,photographs
20,IROO-C3-0001,ERS154526,SAMEA2012637,NEXTGEN,Ovis orientalis,469796,.,Iran,Marand,Marakan,60,male,+45.385152,+38.931678,2011-01-26,.
23,IROO-D6-0003,ERS154528,SAMEA2012639,NEXTGEN,Ovis orientalis,469796,.,Iran,Urmia,Kaboudan,.,male,+45.599,+37.49,2011-02-22,.


Linking ids between datasets:

In [26]:
nextgen2barbato = {}
for sample in plinkio.get_samples():
    if 'C3-0001' in sample or 'D6-0003' in sample:
        nextgen2barbato[sample.replace('MIra', 'IROO')] = sample
nextgen["original_id"] = nextgen["alias"].apply(lambda value: nextgen2barbato[value])
nextgen

,alias,sample_accession,biosamples_id,sample_provider,species,taxonomy_id,breed,country,closest_city,closest_locality,estimated_age_months,sex,longitude,latitude,sampling_date,photographs,original_id
20,IROO-C3-0001,ERS154526,SAMEA2012637,NEXTGEN,Ovis orientalis,469796,.,Iran,Marand,Marakan,60,male,+45.385152,+38.931678,2011-01-26,.,MIra-C3-0001
23,IROO-D6-0003,ERS154528,SAMEA2012639,NEXTGEN,Ovis orientalis,469796,.,Iran,Urmia,Kaboudan,.,male,+45.599,+37.49,2011-02-22,.,MIra-D6-0003


Join metadata table:

In [27]:
muflon_metadata = pd.merge(muflon_metadata, nextgen, on="original_id", how="left")

Now write them into metadata files:

In [28]:
muflon_metadata.to_excel("barbato_muflon_metadata.xlsx", index=False)

Now it's time to take a look to sheep samples:

In [29]:
sheep = pd.read_excel(barbato_2017.working_dir / "41598_2017_7382_MOESM2_ESM/barbato_sheep.xlsx")
sheep

,Breed/population,Acronym,Origin,Number,Ho (SD),Ne,F,Source,Ho
0,Altamurana,ALT,Italy,24,0.16,628,0.06,KJa,0.37
1,Australian Merino,ASM,Spain,24,0.15,920,0.06,KJa,0.37
2,Castellana,CAS,Spain,23,0.16,813,0.02,KJa,0.38
3,Chios,CHI,Greece,23,0.17,391,0.15,KJa,0.33
4,Churra,CHU,Spain,24,0.16,617,0.05,KJa,0.37
5,Comisana,COM,Italy,24,0.16,1028,0.03,KJa,0.38
6,Cyprus Fat Tail,CFT,Cyprus,24,0.19,186,0.13,KJa,0.34
7,Iranian sheep,IRS,Iran,6,0.22,412,0.05,NGb,0.37
8,Milk Lacaune,LAC,France,24,0.16,607,0.06,KJa,0.37
9,Nera di Arbus sheep,SAB,Sardinia,20,0.18,366,0.08,KJa,0.36


Replace origin with the proper country:

In [30]:
sheep.replace({"Origin": {"Sardinia": "Italy", "Corsica": "France", "Iran": "Iran, Islamic Republic of"}}, inplace=True)
sheep

,Breed/population,Acronym,Origin,Number,Ho (SD),Ne,F,Source,Ho
0,Altamurana,ALT,Italy,24,0.16,628,0.06,KJa,0.37
1,Australian Merino,ASM,Spain,24,0.15,920,0.06,KJa,0.37
2,Castellana,CAS,Spain,23,0.16,813,0.02,KJa,0.38
3,Chios,CHI,Greece,23,0.17,391,0.15,KJa,0.33
4,Churra,CHU,Spain,24,0.16,617,0.05,KJa,0.37
5,Comisana,COM,Italy,24,0.16,1028,0.03,KJa,0.38
6,Cyprus Fat Tail,CFT,Cyprus,24,0.19,186,0.13,KJa,0.34
7,Iranian sheep,IRS,"Iran, Islamic Republic of",6,0.22,412,0.05,NGb,0.37
8,Milk Lacaune,LAC,France,24,0.16,607,0.06,KJa,0.37
9,Nera di Arbus sheep,SAB,Italy,20,0.18,366,0.08,KJa,0.36


Many of these samples seems to be already in smarter database as *sheep hapmap data*. Try to filter out the samples I have:

In [31]:
sheep_hapmap = Dataset.objects.get(file="ovine_SNP50HapMap_data.zip")

In [32]:
acronyms = sheep["Acronym"].values
tmp = defaultdict(list)
for fid, iid, *_ in plinkio.read_pedfile():
    if fid in acronyms:
        if SampleSheep.objects.filter(dataset=sheep_hapmap, original_id=iid).count() == 0:
            tmp["code"].append(fid)
            tmp["original_id"].append(iid)
            
tmp = pd.DataFrame(data=tmp)
tmp.head()

,code,original_id
0,SAR,7_C5-1999-A
1,SAR,8_C5-2000-A
2,SAR,9_C5-2001-A
3,SAR,58_PecoraS
4,SAR,59_PecoraS


In [33]:
sheep_metadata = pd.merge(tmp, sheep, left_on="code", right_on="Acronym")[["original_id", "Breed/population", "code", "Origin"]]
sheep_metadata.rename(columns={"Breed/population": "breed", "Origin": "country"}, inplace=True)
sheep_metadata

,original_id,breed,code,country
0,7_C5-1999-A,Sarda sheep,SAR,Italy
1,8_C5-2000-A,Sarda sheep,SAR,Italy
2,9_C5-2001-A,Sarda sheep,SAR,Italy
3,58_PecoraS,Sarda sheep,SAR,Italy
4,59_PecoraS,Sarda sheep,SAR,Italy
5,60_PecoraS,Sarda sheep,SAR,Italy
6,61_PecoraS,Sarda sheep,SAR,Italy
7,67_PecoraS,Sarda sheep,SAR,Italy
8,70_PecoraS,Sarda sheep,SAR,Italy
9,91_PecoraS,Sarda sheep,SAR,Italy


There are also additional metadata coming from nextgen for the some iranian samples:

In [34]:
nextgen = pd.read_table(barbato_2017.working_dir / "41598_2017_7382_MOESM2_ESM/ovis.sample_index", skiprows=10)
nextgen.rename(columns={"#sample_name": "sample_name"}, inplace=True)
sheep_metadata = pd.merge(sheep_metadata, nextgen, left_on="original_id", right_on="sample_name", how="left")
sheep_metadata.tail()

,original_id,breed_x,code,country_x,sample_name,sample_accession,biosamples_id,sample_provider,species,taxonomy_id,breed_y,country_y,closest_city,closest_locality,estimated_age_months,sex,longitude,latitude,sampling_date,photographs
19,IROA-B4-5190,Iranian sheep,IRS,"Iran, Islamic Republic of",IROA-B4-5190,ERS154865,SAMEA2012929,NEXTGEN,Ovis aries,9940.0,.,Iran,salmas,salmas,36,female,+44.838767,+38.153121,2011-10-27,"IROA-B4-5190c.JPG,IROA-B4-5190a.JPG,IROA-B4-51..."
20,IROA-B5-5295,Iranian sheep,IRS,"Iran, Islamic Republic of",IROA-B5-5295,ERS154863,SAMEA2012927,NEXTGEN,Ovis aries,9940.0,.,Iran,urumie,urumie,24,male,+44.952849,+37.972674,2011-10-26,"IROA-B5-5295a.JPG,IROA-B5-5295c.JPG,IROA-B5-52..."
21,IROA-D6-5152,Iranian sheep,IRS,"Iran, Islamic Republic of",IROA-D6-5152,ERS154866,SAMEA2012930,NEXTGEN,Ovis aries,9940.0,.,Iran,ajabshir,ajabshir,24,female,+45.877259,+37.470274,2011-10-25,"IROA-D6-5152a.JPG,IROA-D6-5152c.JPG,IROA-D6-51..."
22,IROA-F3-5142,Iranian sheep,IRS,"Iran, Islamic Republic of",IROA-F3-5142,ERS154867,SAMEA2012931,NEXTGEN,Ovis aries,9940.0,.,Iran,ahar,ahar,48,female,+46.85095,+38.52272,2011-11-15,"IROA-F3-5142c.JPG,IROA-F3-5142a.JPG,IROA-F3-51..."
23,IROA-G4-5205,Iranian sheep,IRS,"Iran, Islamic Republic of",IROA-G4-5205,ERS154862,SAMEA2012926,NEXTGEN,Ovis aries,9940.0,.,Iran,niaz,Meshkin shahr,84,female,+47.430937,+38.3934,2011-11-14,"IROA-G4-5205c.JPG,IROA-G4-5205a.JPG,IROA-G4-52..."


Write metadata to file:

In [35]:
sheep_metadata.to_excel("barbato_sheep_metadata.xlsx", index=False)

<a id='ciani_2020'></a>
## European and balcan sheeps
This dataset comes from [Ciani et al. (2020)](https://doi.org/10.1186/s12711-020-00545-7). I've already processed from supplementary material breeds which are not currently included into SMARTER database, and I've fixed with openrefine the country of origin of each sample.

In [36]:
ciani_2020 = Dataset.objects.get(file="Ciani_2020.zip")
ciani_2020.contents

['8947346/',
 '8947346/12711_2020_545_MOESM2_ESM.xlsx',
 '8947346/OaSNP1477x21960-1807.bim',
 '8947346/OaSNP1477x21960-1807.bed',
 '8947346/OaSNP1477x44430-1807.bim',
 '8947346/OaSNP1477x21960-1807.fam',
 '8947346/OaSNP1477x44430-1807.fam',
 '8947346/12711_2020_545_MOESM1_ESM.xlsx',
 '8947346/OaSNP1477x44430-1807.bed',
 '8947346/ciani_2020_refined.xlsx',
 '8947346/ciani_2020_refined.openrefine.tar.gz',
 '8947346/ciani_2020.xlsx',
 '8947346/ciani_2020_metadata.xlsx']

Ok open dataset and start exploring data:

In [37]:
prefix = str(ciani_2020.working_dir / "8947346/OaSNP1477x44430-1807")
plinkio = CustomBinaryPlinkIO(prefix=prefix, species=ciani_2020.species, chip_name=ciani_2020.chip_name)
plinkio.n_of_individuals = ciani_2020.n_of_individuals

In [38]:
plinkio.read_mapfile()
plinkio.fetch_coordinates(src_assembly=OAR3)

In [39]:
snps_found = len(plinkio.mapdata)-len(plinkio.filtered)
perc_missing = round(100 - (snps_found / len(plinkio.mapdata) * 100), 2)

print(f"I can retrieve {snps_found} of {len(plinkio.mapdata)} SNPs using 'name' ({perc_missing}% missing)")

I can retrieve 44430 of 44430 SNPs using 'name' (0.0% missing)


Is this dataset in top coordinates?

In [40]:
plinkio.is_top()

  0%|          | 0/1477 [00:00<?, ?it/s]

True

Ok, it's in top coordinartes. Now check breeds and metadata and try to define a new metadata table to be assigned to each sample. Some samples are already in smarter database:

In [41]:
ciani_2020_sheeps = pd.read_excel(ciani_2020.working_dir / "8947346/ciani_2020_refined.xlsx")
ciani_2020_sheeps.head()

,country,region,breed,fid,Code,Type,1477 sheep,2121 sheep,"44,430 SNPs","21,960 SNPs",Source of data,latitude,longitude,Northern latitudea,Eastern longitudea,Breed (country of sampling)b,country_old,region_old
0,Iran,Southwestern Asia,Lori-Bakhtiari,LoriBakhtiari,LBA,Fat-tailed,24.0,46,0.360253,0.386850,[37],33.480000,48.350000,3348.000000,4835.000000,Lori-Bakhtiari,Iran,Southwestern Asia
1,Iran,Southwestern Asia,Zel,Zel,ZEL,Thin-tailed,24.0,47,0.359735,0.385835,[37],36.550000,53.100000,3655.000000,5310.000000,Zel,Iran,Southwestern Asia
2,Kazakhstan,Southwestern Asia,Karakul,Karakul,KAR,Fat-tailed,6.0,6,0.355527,0.378244,This study,40.170000,63.670000,4017.000000,6367.000000,Karakul (Romania),Kazachstan,Southwestern Asia
3,Greece,East-Aegean,Kymi,Kymi,KYM,Thin-tail,6.0,6,0.361707,0.383257,This study,38.501745,24.109574,3850.174490,2410.957407,Kymi,Greece,East-Aegean
4,Greece,East-Aegean,Lesvos,Lesvos,LES,Semi-fat-tailed,6.0,6,0.367334,0.391434,This study,39.229218,26.193922,3922.921808,2619.392158,Lesvos,Greece,East-Aegean


Now read *FID* and *IID* from plink file:

In [42]:
tmp = defaultdict(list)
for fid, iid, *_ in plinkio.read_pedfile():
    tmp["code"].append(fid)
    tmp["original_id"].append(iid)
            
tmp = pd.DataFrame(data=tmp)
tmp.head()

,code,original_id
0,Finn,FIN5
1,Finn,FIN13
2,Finn,FIN21
3,Finn,FIN24
4,Finn,FIN25


Ok, now merge tables:

In [43]:
ciani_2020_metadata = pd.merge(tmp, ciani_2020_sheeps, left_on="code", right_on="fid")
ciani_2020_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 910 entries, 0 to 909
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   code                          910 non-null    object 
 1   original_id                   910 non-null    object 
 2   country                       910 non-null    object 
 3   region                        910 non-null    object 
 4   breed                         910 non-null    object 
 5   fid                           910 non-null    object 
 6   Code                          910 non-null    object 
 7   Type                          745 non-null    object 
 8   1477 sheep                    910 non-null    float64
 9   2121 sheep                    910 non-null    int64  
 10  44,430 SNPs                   910 non-null    float64
 11  21,960 SNPs                   910 non-null    float64
 12  Source of data                838 non-null    object 
 13  latit

Drop columns I don't need:

In [44]:
ciani_2020_metadata.drop([
    "code", "Northern latitudea", "Eastern longitudea", 
    "Breed (country of sampling)b", "country_old", "region_old",
    "1477 sheep", "2121 sheep", "44,430 SNPs", "21,960 SNPs"
], axis=1, inplace=True)
ciani_2020_metadata.rename(columns={"Code": "code"}, inplace=True)
ciani_2020_metadata.head()

,original_id,country,region,breed,fid,code,Type,Source of data,latitude,longitude
0,NWI1001,Norway,North-continental,Norwegian White,NorwegianWhite,NWI,Thin-tailed,[76],62.6,9.67
1,NWI1002,Norway,North-continental,Norwegian White,NorwegianWhite,NWI,Thin-tailed,[76],62.6,9.67
2,NWI1003,Norway,North-continental,Norwegian White,NorwegianWhite,NWI,Thin-tailed,[76],62.6,9.67
3,NWI1004,Norway,North-continental,Norwegian White,NorwegianWhite,NWI,Thin-tailed,[76],62.6,9.67
4,NWI1005,Norway,North-continental,Norwegian White,NorwegianWhite,NWI,Thin-tailed,[76],62.6,9.67


How many breeds are currently in this dataset?

In [45]:
ciani_2020_metadata["breed"].value_counts()

Croatian Isles       90
Bergamasca           24
Appenninica          24
Sardinian mouflon    24
Laticauda            24
                     ..
Sumavska              4
Lika                  4
Dubska                2
Privorska             2
Recka                 2
Name: breed, Length: 61, dtype: int64

What about species? get muflon breeds:

In [46]:
ciani_2020_metadata[ciani_2020_metadata["breed"].str.contains("mouflon", case=False)]["breed"].value_counts()

Sardinian mouflon    24
European mouflon     21
Iranian mouflon      14
Name: breed, dtype: int64

Ok try to add a species column:

In [47]:
def get_species(breed):
    if breed in ["Sardinian mouflon", "European mouflon"]:
        return "Ovis aries musimon"
    elif breed == "Iranian mouflon":
        return "Ovis orientalis"
    else:
        return "Ovis aries"
    
ciani_2020_metadata["species"] = ciani_2020_metadata["breed"].apply(get_species)
ciani_2020_metadata.head()

,original_id,country,region,breed,fid,code,Type,Source of data,latitude,longitude,species
0,NWI1001,Norway,North-continental,Norwegian White,NorwegianWhite,NWI,Thin-tailed,[76],62.6,9.67,Ovis aries
1,NWI1002,Norway,North-continental,Norwegian White,NorwegianWhite,NWI,Thin-tailed,[76],62.6,9.67,Ovis aries
2,NWI1003,Norway,North-continental,Norwegian White,NorwegianWhite,NWI,Thin-tailed,[76],62.6,9.67,Ovis aries
3,NWI1004,Norway,North-continental,Norwegian White,NorwegianWhite,NWI,Thin-tailed,[76],62.6,9.67,Ovis aries
4,NWI1005,Norway,North-continental,Norwegian White,NorwegianWhite,NWI,Thin-tailed,[76],62.6,9.67,Ovis aries


There are the croatian breeds which have multiple entries caused by the table joins

In [48]:
vc = ciani_2020_metadata["original_id"].value_counts()
croatian_isles = ciani_2020_metadata[ciani_2020_metadata["original_id"].isin(vc[vc > 1].index)].copy()
croatian_isles.head()

,original_id,country,region,breed,fid,code,Type,Source of data,latitude,longitude,species
601,KRK17,Croatia,Balkans,Croatian Isles,CroatianIsles,CRI,"Zackel, Pramenka",This study,44.840000,14.680000,Ovis aries
602,KRK17,Croatia,Cres Island,Croatian Isles,CroatianIsles,CRI,"Zackel, Pramenka",NaN,45.007050,14.396897,Ovis aries
603,KRK17,Croatia,Krk Island,Croatian Isles,CroatianIsles,CRI,"Zackel, Pramenka",NaN,45.080000,14.592586,Ovis aries
604,KRK17,Croatia,Pag Island,Croatian Isles,CroatianIsles,CRI,"Zackel, Pramenka",NaN,44.496582,14.950000,Ovis aries
605,KRK17,Croatia,Rab Island,Croatian Isles,CroatianIsles,CRI,"Zackel, Pramenka",NaN,44.770698,14.765352,Ovis aries


The first three letters of `original_id` are the first tree letter of the region:

In [49]:
def isin_island(original_id, region):
    pattern = original_id[:3].lower()
    if pattern in region.lower():
        return True
    else:
        return False
    
to_remove = croatian_isles[~np.vectorize(isin_island)(croatian_isles["original_id"], croatian_isles["region"])].index

Drop indexes from table and write metadata file:

In [50]:
ciani_2020_metadata.drop(index=to_remove).to_excel("ciani_2020_metadata.xlsx", index=False)

<a id='northwest_africa'></a>
## Northwest Africa sheeps
This dataset comes from [Belabdi et al. (2019)](https://doi.org/10.1038/s41598-019-44137-y). There's a metadata xls file which need to be fixed before importing this dataset

In [51]:
belabdi_2019 = Dataset.objects.get(file="northwest_africa_sheep.zip")
belabdi_2019.contents

['northwest_africa_sheep/',
 'northwest_africa_sheep/belabdi_2019_metadata.xlsx',
 'northwest_africa_sheep/41598_2019_44137_MOESM1_ESM.pdf',
 'northwest_africa_sheep/README_for_AlgerianSheepSidaounHamra.xlsx',
 'northwest_africa_sheep/AlgerianSheepSidaounHamra.ped',
 'northwest_africa_sheep/AlgerianSheepSidaounHamra.map']

Ok open dataset and start exploring data:

In [52]:
prefix = str(belabdi_2019.working_dir / "northwest_africa_sheep/AlgerianSheepSidaounHamra")
plinkio = CustomTextPlinkIO(prefix=prefix, species=belabdi_2019.species, chip_name=belabdi_2019.chip_name)
plinkio.n_of_individuals = belabdi_2019.n_of_individuals

In [53]:
plinkio.read_mapfile()
plinkio.fetch_coordinates(src_assembly=OAR3)

In [54]:
snps_found = len(plinkio.mapdata)-len(plinkio.filtered)
perc_missing = round(100 - (snps_found / len(plinkio.mapdata) * 100), 2)

print(f"I can retrieve {snps_found} of {len(plinkio.mapdata)} SNPs using 'name' ({perc_missing}% missing)")

I can retrieve 51135 of 51135 SNPs using 'name' (0.0% missing)


Is this dataset in top coordinates?

In [55]:
plinkio.is_top()

  0%|          | 0/48 [00:00<?, ?it/s]

True

Ok, even this is in *TOP*. Let explore plink file:

In [56]:
tmp = defaultdict(list)
for fid, iid, *_ in plinkio.read_pedfile():
    tmp["code"].append(fid)
    tmp["original_id"].append(iid)
            
tmp = pd.DataFrame(data=tmp)
tmp.head()

,code,original_id
0,1,S50091
1,2,S50092
2,3,S50093
3,4,S50094
4,5,S50095


The point is that the breed is *progressive numerical*: I can change it or map *fid* into metadata. Now read metadata and try to fix stuff:

In [57]:
belabdi_2019_sheeps = pd.read_excel(belabdi_2019.working_dir / "northwest_africa_sheep/README_for_AlgerianSheepSidaounHamra.xlsx")
belabdi_2019_sheeps.head()

,code,Sample ID,Call Rate,Identification,breed,Longitude,Latitude,sexe,age
0,S50091,S50091,0.993644,H01,Hamra,"0°46'0.80""O","33°18'6.59""N",F,3
1,S50092,S50092,0.995131,H02,Hamra,"0° 7'6.30""E","34°45'30.07""N",F,4
2,S50093,S50093,0.988657,H03,Hamra,"0° 3'23.08""E","34°37'14.41""N",F,3
3,S50094,S50094,0.994739,H06,Hamra,"0°36'9.46""O","32°45'1.79""N",F,4
4,S50095,S50095,0.995600,H07,Hamra,"0°42'7.08""O","32°42'16.82""N",F,5


Well, we need to fix up *latitude* and *longitude* to determine country location:

In [58]:
def coordinate_converter(longitude: str, latitude: str):
    tmp = " ".join([f"{tmp[-1]} {tmp[:-1]}" for tmp in [longitude.strip(), latitude.strip()]])
    return Point(tmp)

In [59]:
points = belabdi_2019_sheeps[['Longitude', 'Latitude']].apply(lambda df: coordinate_converter(df['Longitude'], df['Latitude']), axis=1)
belabdi_2019_sheeps["latitude"] = points.apply(lambda point: point.latitude)
belabdi_2019_sheeps["longitude"] = points.apply(lambda point: point.longitude)
belabdi_2019_sheeps.head()

,code,Sample ID,Call Rate,Identification,breed,Longitude,Latitude,sexe,age,latitude,longitude
0,S50091,S50091,0.993644,H01,Hamra,"0°46'0.80""O","33°18'6.59""N",F,3,0.766889,33.301831
1,S50092,S50092,0.995131,H02,Hamra,"0° 7'6.30""E","34°45'30.07""N",F,4,0.118417,34.758353
2,S50093,S50093,0.988657,H03,Hamra,"0° 3'23.08""E","34°37'14.41""N",F,3,0.056411,34.620669
3,S50094,S50094,0.994739,H06,Hamra,"0°36'9.46""O","32°45'1.79""N",F,4,0.602628,32.750497
4,S50095,S50095,0.995600,H07,Hamra,"0°42'7.08""O","32°42'16.82""N",F,5,0.701967,32.704672


Now, determine country with *reverse geocoding*:

In [60]:
locator = Nominatim(user_agent="myGeocoder", timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)

def get_country(coordinate):
    data = rgeocode(coordinate, language="English")
    if data:
        country_code = data.raw['address']['country_code']
        return pycountry.countries.get(alpha_2=country_code).name
    else:
        return data

belabdi_2019_sheeps["coordinates"] = belabdi_2019_sheeps["latitude"].map(str) + "," + belabdi_2019_sheeps["longitude"].map(str)
belabdi_2019_sheeps["country"] = belabdi_2019_sheeps["coordinates"].progress_apply(get_country)
belabdi_2019_sheeps.head()

  0%|          | 0/48 [00:00<?, ?it/s]

,code,Sample ID,Call Rate,Identification,breed,Longitude,Latitude,sexe,age,latitude,longitude,coordinates,country
0,S50091,S50091,0.993644,H01,Hamra,"0°46'0.80""O","33°18'6.59""N",F,3,0.766889,33.301831,"0.766888888888889,33.301830555555554",Uganda
1,S50092,S50092,0.995131,H02,Hamra,"0° 7'6.30""E","34°45'30.07""N",F,4,0.118417,34.758353,"0.11841666666666667,34.75835277777778",Kenya
2,S50093,S50093,0.988657,H03,Hamra,"0° 3'23.08""E","34°37'14.41""N",F,3,0.056411,34.620669,"0.05641111111111111,34.620669444444445",Kenya
3,S50094,S50094,0.994739,H06,Hamra,"0°36'9.46""O","32°45'1.79""N",F,4,0.602628,32.750497,"0.6026277777777778,32.75049722222222",Uganda
4,S50095,S50095,0.995600,H07,Hamra,"0°42'7.08""O","32°42'16.82""N",F,5,0.701967,32.704672,"0.7019666666666666,32.70467222222222",Uganda


Try to join metadata table with *plink* file information

In [61]:
belabdi_2019_metadata = pd.merge(tmp, belabdi_2019_sheeps, left_on="original_id", right_on="Sample ID")
belabdi_2019_metadata.head()

,code_x,original_id,code_y,Sample ID,Call Rate,Identification,breed,Longitude,Latitude,sexe,age,latitude,longitude,coordinates,country
0,1,S50091,S50091,S50091,0.993644,H01,Hamra,"0°46'0.80""O","33°18'6.59""N",F,3,0.766889,33.301831,"0.766888888888889,33.301830555555554",Uganda
1,2,S50092,S50092,S50092,0.995131,H02,Hamra,"0° 7'6.30""E","34°45'30.07""N",F,4,0.118417,34.758353,"0.11841666666666667,34.75835277777778",Kenya
2,3,S50093,S50093,S50093,0.988657,H03,Hamra,"0° 3'23.08""E","34°37'14.41""N",F,3,0.056411,34.620669,"0.05641111111111111,34.620669444444445",Kenya
3,4,S50094,S50094,S50094,0.994739,H06,Hamra,"0°36'9.46""O","32°45'1.79""N",F,4,0.602628,32.750497,"0.6026277777777778,32.75049722222222",Uganda
4,5,S50095,S50095,S50095,0.995600,H07,Hamra,"0°42'7.08""O","32°42'16.82""N",F,5,0.701967,32.704672,"0.7019666666666666,32.70467222222222",Uganda


Try to add a *code* column:

In [62]:
def breed2code(breed):
    if breed == "Sidaoun":
        return "SDN"
    elif breed == "Hamra":
        return "HMR"
    else:
        raise Exception("Breed not found!")
        
belabdi_2019_metadata["code"] = belabdi_2019_metadata["breed"].apply(breed2code)
belabdi_2019_metadata.head()

,code_x,original_id,code_y,Sample ID,Call Rate,Identification,breed,Longitude,Latitude,sexe,age,latitude,longitude,coordinates,country,code
0,1,S50091,S50091,S50091,0.993644,H01,Hamra,"0°46'0.80""O","33°18'6.59""N",F,3,0.766889,33.301831,"0.766888888888889,33.301830555555554",Uganda,HMR
1,2,S50092,S50092,S50092,0.995131,H02,Hamra,"0° 7'6.30""E","34°45'30.07""N",F,4,0.118417,34.758353,"0.11841666666666667,34.75835277777778",Kenya,HMR
2,3,S50093,S50093,S50093,0.988657,H03,Hamra,"0° 3'23.08""E","34°37'14.41""N",F,3,0.056411,34.620669,"0.05641111111111111,34.620669444444445",Kenya,HMR
3,4,S50094,S50094,S50094,0.994739,H06,Hamra,"0°36'9.46""O","32°45'1.79""N",F,4,0.602628,32.750497,"0.6026277777777778,32.75049722222222",Uganda,HMR
4,5,S50095,S50095,S50095,0.995600,H07,Hamra,"0°42'7.08""O","32°42'16.82""N",F,5,0.701967,32.704672,"0.7019666666666666,32.70467222222222",Uganda,HMR


Remove and rename columns. Write to *xlsx* file:

In [63]:
belabdi_2019_metadata.drop(
    ["coordinates", "Latitude", "Longitude", "code_y", "Sample ID"], axis=1
).rename(
    columns={"Call Rate": "call_rate", "code_x": "fid", "Identification": "identification"}
).to_excel("belabdi_2019_metadata.xlsx", index=False)